In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

<img src="../../../common/logo_DH.svg" align='left' width=50%/>

# LAB: KNN

## 1. Introducción

El objetivo de este ejercicio es clasificar si un determinado tipo de vino es de alta o baja calidad. Para eso usaremos un dataset que contiene un set amplio de _features_ vinculados a diversas características del vino, tales como acidez, azúclar, densidad, ph, si es tinto, etc.

Usaremos como target high_quality, una discretización de la variable quality.

Comencemos leyendo los datos...

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../Data/wine.csv')
df.head()

## 2. Análisis exploratorio

- ¿Cuántas observaciones hay en la tabla? ¿Tenemos datos faltantes?
- ¿Cómo se distribuye la variable target?
- ¿Existen variables redundantes?
- ¿Qué variables están más correlacionadas con el _target_?
- ¿Cómo podemos visualizar las relaciones entre cada par de variables?

In [ ]:
# ¿Cuántas observaciones hay en la tabla? ¿Tenemos datos faltantes?
df.info()

In [ ]:
# ¿Cómo se distribuye la variable target?
df['high_quality'].value_counts(normalize=True)

In [ ]:
# ¿Existen variables redundantes?
# high_quality es una binarización de la variable quality
df.groupby('quality')['high_quality'].mean()

In [ ]:
# is_red es una binarización de la variable color
df.groupby('color')['is_red'].mean()

In [ ]:
# ¿Qué variables están más correlacionadas con el target?
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20,10))
sns.heatmap(df.corr(), annot=True, vmin=-1, cmap='Blues');

In [ ]:
# ¿Cómo podemos visualizar las relaciones entre cada par de variables?
sns.pairplot(df);

## 3. Preprocesamiento y limpieza del dataset

- Construir la matriz de _features_, prestando especial atención a no incluir las variables redundantes ni replicar información del _target_ en ninguna _feature_ (¡siempre hay que evitar el filtrado de información!)
- Construir la variable _target_: high_quality
- Separar los conjuntos de entrenamiento y testeo, estratificando por clase
- ¿Necesitamos estandarizar las variables?

In [ ]:
# Construir la matriz de features
X = df.drop(['color', 'quality', 'high_quality'], axis=1)
X.head()

In [ ]:
# Construir la variable target: high_quality
y = df['high_quality']
y.head()

In [ ]:
# Separar los conjuntos de entrenamiento y testeo, estratificando por clase
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [ ]:
# ¿Necesitamos estandarizar las variables?
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

## 4. Entrenando KNN

- Experimentar con diferentes valores para el hiperparámetro _k_ y evaluar el accuracy promedio del modelo y su dispersión en validación cruzada sobre el _traininig set_
- Visualizar los resultados

**Pista:** pueden tratar de generar una función que encapsule la generación de la tabla de resultados vimos en la notebook 1.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def scores_knn(X, y, start,stop,step):
    
    # Vamos a querer graficar los distintos valores del score de cross validation en función del hiperparámetro n_neighbors
    # Para esto vamos a generar una lista de diccionarios que después se puede convertir fácilmente en DataFrame
    
    # Lista de diccionarios - la inicializamos vacío y por fuera del for loop para ir alimentándola en cada iteración
    scores_para_df = []
    
    
    for i in range(start,stop,step):
        
        # En cada iteración, instanciamos el modelo con un hiperparámetro distinto
        model = KNeighborsClassifier(n_neighbors=i)

        # cross_val_scores nos devuelve un array de 5 resultados, uno por cada partición que hizo automáticamente CV
        kf = KFold(n_splits=10, shuffle=True, random_state=10)
        cv_scores = cross_val_score(model, X, y, cv=kf)

        # Para cada valor de n_neighbours, creamos un diccionario con el valor de n_neighbours y la media y el desvío de los scores
        dict_row_score = {'score_medio':np.mean(cv_scores),'score_std':np.std(cv_scores),'n_neighbours':i}

        # Guardamos cada uno en la lista de diccionarios
        scores_para_df.append(dict_row_score)
    
    # Creamos el DF a partir de la lista de resultados
    df_scores = pd.DataFrame(scores_para_df)
    
    # Incorporamos los límites inferior y superior, restando y sumando el valor del desvío estándar, respectivamente
    df_scores['limite_inferior'] = df_scores['score_medio'] - df_scores['score_std']
    df_scores['limite_superior'] = df_scores['score_medio'] + df_scores['score_std']
    
    # Retornamos el DF
    return df_scores

In [ ]:
# Probamos de 1 a 20 vecinos
df_scores= scores_knn(X_train, y_train, 1, 21, 1)

In [ ]:
# Visualizamos los resultados

plt.plot(df_scores['n_neighbours'], df_scores['limite_inferior'], color='r')
plt.plot(df_scores['n_neighbours'], df_scores['score_medio'], color='b')
plt.plot(df_scores['n_neighbours'], df_scores['limite_superior'], color='r')
plt.ylim(0.7, 1);

## 5. Prediciendo sobre _test_

- Habiendo identificado la mejor configuración del modelo en el paso anterior, reentrenar sobre todo el _training set_ y predecir sobre _test_
- ¿Qué accuracy obtenemos sobre _test_? ¿Cómo podemos saber si estamos ante un caso de sub-ajuste o sobre-ajuste?
- Graficar la matriz de confusión

In [ ]:
# Identificamos el score máximo
df_scores.loc[df_scores.score_medio == df_scores.score_medio.max()]

In [ ]:
# Asignamos el valor del k óptimo a una variable
best_k = df_scores.loc[df_scores.score_medio == df_scores.score_medio.max(),'n_neighbours'].values[0]
best_k

In [ ]:
# Elegimos el modelo óptimo que nos había indicado cross validation
model = KNeighborsClassifier(n_neighbors=best_k)

# Lo ajustamos sobre datos de entrenamiento
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

# Evaluamos qué accuracy obtenemos en train
accuracy_score(y_train, model.predict(X_train))

In [ ]:
# Lo utilizamos para predecir en test
y_pred = model.predict(X_test)

In [ ]:
# Computamos el accuracy score en test
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

# Graficamos la matriz de confusión
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='.0f')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');